# Homography decomposition 
This notebook visualizes the decomposition of a homography into relative poses of cameras viewing the same plane.

**Subjects covered**
1. **Definition of homography and its uses**
2. **Using DLT to compute a homography from point correspondences**
3. **Proof of homography decomposition into poses**
4. **Conclusion**
5. **Sources**

**Why are we interested in these subjects?**       
When learning about computer vision, the concept of a homography show up early and often. Homographies appear in many computer vision algorithms, toolsets, and theoretical proofs. They are often used as a tool to map planes to other planes and manipulate quadrilateral shapes. A practical example is the use of a homography to transform a picture of a magazine on a table into a "perspectiveless" frontal view of the magazine. 


Similar to the fundamental matrix, a homography enables us to infer relative poses of cameras. If two cameras picture the same plane from different positions, the homography between the two images can be decomposed into the cameras' relative translation and rotation. Knowing the relative pose will allow for a more targeted search of point correspondences between images. It can also serve as an initial pose estimate when using iterative non-linear pose estimation methods (e.g. PnP Levenberg Marquardt in notebook 2).

**Disclaimer**    
This notebook is mostly intended as thorough reproduction of the decomposition presented by Faugeras and Lustman in their paper [Motion and structure from motion in a piecewise planar environment](https://hal.inria.fr/file/index/docid/75698/filename/RR-0856.pdf). This notebook does provide definitions and simple examples of homographies. Nevertheless, those unfamiliar with homographies are advised to read more accessible resources on homographies, such as: [this tutorial with code examples](https://docs.opencv.org/4.x/d9/dab/tutorial_homography.html) by OpenCV, [these slides](http://www.cse.psu.edu/~rtc12/CSE486/lecture16.pdf) by Robert Collins, [these slides](http://www.cs.columbia.edu/~allen/F17/NOTES/homography_pka.pdf) collected by Peter Allen, [these linear algebra videos on linear transformations](https://www.3blue1brown.com/topics/linear-algebra) by Grant Sanderson. 

## Homography     
#### Defintion
A homography is an isomorphism: a structure-preserving mapping between two structures. Projective spaces are the structures that homographies operate on. For a projective space $\mathbb{P}^2$ it preserves concurrency, collinearity, cross-ratio and more (see [Multiple view geometry](https://www.robots.ox.ac.uk/~vgg/hzbook/) Ch.2.4.7). One may be familiar with projective spaces through the use of scale invariant homogenous 3 vectors $\begin{bmatrix} x\\ y\\ w \end{bmatrix}$ to represent quantities usually expressed as Euclidean 2 vectors $\begin{bmatrix} x\\ y \end{bmatrix}$ in computer vision. Here, scale invariance refers to the equivalence relation between scaled versions of the vector $\begin{bmatrix} x\\ y\\ w \end{bmatrix} = k \begin{bmatrix} x\\ y\\ w \end{bmatrix} \hspace{0.5em} \forall k $. A homography $\mathbf{H}$ operates on these projective points in these projective spaces. For example, $\mathbf{H}$ takes a set of points $x_i$ in $\mathbb{P}^2$ and maps them to a corresponding set of points $x'_i$ likewise in $\mathbb{P}^2$. That is,       

$$ \begin{align} 
    \mathbf{H}&: x \rightarrow x' \\ \\
         x'_1 &= \mathbf{H} \hspace{0.1em} x_1 \end{align}$$

The mapping preserves collinearity by definition, i.e. images of collinear points are themselves collinear i.e. straight lines are mapped to straight lines. A mapping with this property is called a collineation. A collineation is a linear transformation that preserves vector addition and scalar multiplication and can be implemented by a matrix:  


$$ x'_i = \begin{bmatrix} h_{1} & h_{2} & h_{3} \\ 
                          h_{4} & h_{5} & h_{6} \\ 
                          h_{7} & h_{8} & h_{9}\end{bmatrix} \hspace{0.1em} x_1 $$
#### Degrees of freedom
A homography has $8$ degrees of freedom. While its matrix may suggest $9$, the scale invariance of $\mathbb{P}^2$ implies $\mathbf{H} = k \hspace{0.1em}\mathbf{H} \hspace{0.5em}\forall k $ which deducts a degree of freemdom, similar to how a point in $\mathbb{P}^2$ has $2$ degrees of freedom while it has $3$ parameters.  

Besides homographies, there are other collineations with fewer degrees of freedom, each preserving a greater set of properties.

<center> 
<img src="./images/collineations.png" width=600 />   
<br>
Figure taken from Zisserman, Hartley, Multiple view geometry in computer vision
</center>


## Interactive homography
Here we show an interactive widget that visualizes the different types of transformations. 

In [1]:
import numpy as np 
from bqplot import pyplot as plt
from IPython.display import display
from ipywidgets import FloatSlider, IntSlider, HBox, VBox, Layout, HTML, ToggleButtons, Button, Output, Tab
from numpy import cos, sin

In [2]:
# Callback functions. 
# NOTE: All functions with parameters `args*` reference global variables. 
# I know this is ugly; ipywidgets does not user defined parameters in callbacks.

def set_slider_visibility(desired_sliders, all_sliders):   
    """ Updates the sliders exposed to a user after button toggle. """
    for slider in all_sliders:
        if slider in desired_sliders:
            slider.layout.visibility = "visible"
        else: 
            slider.layout.visibility = "hidden"

def on_transform_click(*args):
    """ Sets appropriate displays when transformation type toggle button is clicked. """
    transform = transform_buttons.value
    desired_sliders = slider_map[transform]
    set_slider_visibility(desired_sliders, all_sliders)
    html_matrix_1.value = get_symbol_matrix_html(transform, all_sliders)
    html_matrix_2.value = get_symbol_matrix_html(transform, all_sliders, substitute=True) 
    on_slider_drag()

def on_slider_drag(*args):
    """ Sets appropriate displays when a value slider is dragged. """
    scatter_orig = transform_fig_1.marks[0]
    lines_orig = transform_fig_1.marks[1]
    scatter_trans = transform_fig_2.marks[0]
    lines_trans = transform_fig_2.marks[1]
    matrix_values = get_matrix_values(transform_buttons.value, all_sliders)

    # Update matrix representation
    html_matrix_2.value = get_symbol_matrix_html(transform_buttons.value, all_sliders, substitute=True) 
    html_matrix_3.value = get_value_matrix_html(matrix_values)
    
    # Update transformation visualization
    with scatter_trans.hold_sync(), lines_trans.hold_sync():
        xs, ys, zs = matrix_values @ np.vstack([scatter_orig.x, scatter_orig.y, np.ones_like(scatter_orig.x)])
        zs += 0.001  # avoid 0 div
        scatter_trans.x, scatter_trans.y = xs/zs, ys/zs
        
        # xs, ys, zs = matrix_values @ np.vstack([lines_orig.x, lines_orig.y, np.ones_like(lines_orig.x)])
        # zs += 0.001
        lines_trans.x, lines_trans.y = xs/zs, ys/zs
        
def on_reset_click(*args): 
    """ Resets slider values when reset button is clicked. """
    for slider in all_sliders: 
        slider.value = slider.default_value

In [3]:
# Some verbose formatting functions are placed in notebook_functions.py to save visual space
from notebook_functions import (get_matrix_values, 
                                get_symbol_matrix_html, 
                                get_value_matrix_html, 
                                get_quad_fig, get_html)

#Parameter sliders 
rot_angle = FloatSlider(min=-np.pi, max=np.pi, step=0.005, description=r'rotation \(\theta\)')
x_offset = FloatSlider(min=-5, max=5, step=0.005, description=r'x offset  \(t_x\)')
y_offset = FloatSlider(min=-5, max=5, step=0.005,  description=r'y offset \(t_y\)')
scale = FloatSlider(min=-5, max=5, step=0.005, value=1, description=r'scale \(s\)')
x_scale = FloatSlider(min=-5, max=5, step=0.005, value=1, description=r'x scale \(s_x\)')
y_scale = FloatSlider(min=-5, max=5, step=0.005, value=1, description=r'y scale  \(s_y\)')
x_shear = FloatSlider(min=-5, max=5, step=0.005, description=r'x shear \(\alpha_x\)')
y_shear = FloatSlider(min=-5, max=5, step=0.005, description=r'y shear \(\alpha_y\)')
x_perspec = FloatSlider(min=-0.5, max=0.5, step=0.005, description=r'x perspective \(d_x\)', style={'description_width': 'initial'})
y_perspec = FloatSlider(min=-0.5, max=0.5, step=0.005, description=r'y perspective \(d_y\)', style={'description_width': 'initial'})

# Collections of parameter sliders belonging to transformations
euclidean = [rot_angle, x_offset, y_offset]
similarity = euclidean + [scale]
affine = euclidean + [x_shear, y_shear, x_scale, y_scale]
projective = affine + [x_perspec, y_perspec]
all_sliders = [rot_angle, x_offset, y_offset, scale, x_scale, y_scale, x_shear, y_shear, x_perspec, y_perspec]

# Set default values property for easy resetting
for slider in all_sliders:
    slider.default_value = slider.value

# Map from button names to slider sets
slider_map = {'Euclidean': euclidean, 
              'Similarity': similarity, 
              'Affine': affine,
              'Projective': projective}

# Initialize Widgets
default_transform = 'Projective'
matrix_values = get_matrix_values(default_transform, all_sliders)
html_matrix_1 = HTML(value=get_symbol_matrix_html(default_transform, all_sliders))
html_matrix_2 = HTML(value=get_symbol_matrix_html(default_transform, all_sliders, substitute=True))
html_matrix_3 = HTML(value=get_value_matrix_html(matrix_values))
reset_button = Button(description='Reset', button_style='info')
transform_buttons = ToggleButtons(options=['Projective', 'Affine', 'Similarity', 'Euclidean'])
transform_fig_1 = get_quad_fig('Original')
transform_fig_2 = get_quad_fig('Transformed')
transform_fig_1.background_style = {"fill": '#fafafa'}
set_slider_visibility(slider_map[default_transform], all_sliders)

# Set interactivitiy callbacks
reset_button.on_click(on_reset_click)
transform_buttons.observe(on_transform_click)
for slider in all_sliders: 
    slider.observe(on_slider_drag)

# Build UI by stacking ipywidgets
sliders_1 = VBox(all_sliders[:4])
sliders_2 = VBox(all_sliders[4:8])
sliders_3 = VBox(all_sliders[8:])
sliders_ui = HBox([sliders_1, sliders_2, sliders_3], layout=Layout(margin='20px'))
math_ui = HBox([html_matrix_1, html_matrix_2, html_matrix_3])
transform_ui = HBox([transform_fig_1, transform_fig_2])
transform_buttons_ui = HBox([transform_buttons, reset_button])
ui = VBox([VBox([transform_buttons_ui, sliders_ui, math_ui]), transform_ui])
display(ui)

## Direct Linear Transform (DLT) - Homography from point correspondences

The Direct Linear Transform (DLT) algorithm is similar to the 8 point algorithm used to estimate the fundamental matrix (in notebook 3). 
The relationship between point sets defined by a homography $x' = \mathbf{H} \hspace{0.1em} x$ is used to build linear constraints from point correspondence . 
Each point correspondence provides 2 constraints. To solve for the 8 degrees of freedom of a homography we require 4 point correspondences. The approach is as follows: 

* Find 4 or more point correspondences and build constraints. Here we will use chessboard points. 
* Use an SVD to find the least-squares solution to $\mathbf{H}$ given the constraints $x' = \mathbf{H}\hspace{0.1em} x$. That is, for SVD$= \mathbf {U\Sigma V^{T}}$, take the last column of $\mathbf{V}$ as $\mathbf{H}$. *
   
*-See the appendix of notebook 3 for an intuitive explanation and proof of why the Singular Value Decomposition can be used to find a least-squares solution.

$\begin{align}
x' &= \mathbf{H} \hspace{0.1em} x  \hspace{0.5em}
\text{For a single piont correspondence }\\ \\
\begin{bmatrix}
x' \\ y' \\ w'
\end{bmatrix} &= 
\begin{bmatrix}
h_1 & h_2 & h_3 \\
h_4 & h_5 & h_6 \\
h_7 & h_8 & h_9
\end{bmatrix}  
\begin{bmatrix}
x \\ y \\ w
\end{bmatrix} \\ \\
x' &=  \frac{h_1 x + h_2 y + h_3}{h_7 x + h_8 y + h_9}  \hspace{0.5em}
\text{Given } w=1 \\ \\
y' &=  \frac{h_4 x + h_5 y + h_6}{h_7 x + h_8 y + h_9} \\
\\
x'(h_7 x + h_8 y + h_9) - h_1 x - h_2 y - h_3 &= 0 \\ \\
y'(h_4 x + h_5 y + h_6) - h_7 x - h_8 y - h_9 &= 0 \\ \\
\\
\\
\begin{bmatrix}
-x & -y & -1 & 0 & 0 & 0 & xx' & yx' & x' \\
 0 & 0 & 0 & -x & -y & -1 & xy' & yy' & y'
\end{bmatrix}
\begin{bmatrix}
h_1 \\ h_2 \\ h_3 \\ h_4 \\ h_5 \\ h_6 \\ h_7 \\ h_8 \\ h_9
\end{bmatrix} &= \begin{bmatrix} 0 \\ 0 \end{bmatrix}
\end{align}$




In [ ]:
  def compute_homography(points_1, points_2):
    constraint_matrix = list()
    for point_1, point_2 in zip(points_1.T, points_2.T): 
        x1, y1 = point_1[:2]
        x2, y2 = point_2[:2]
        constraint_1 = [-x1, -y1, -1,   0,   0,  0, x1 * x2, y1 * x2, x2]
        constraint_2 = [  0,   0,  0, -x1, -y1, -1, x1 * y2, y1 * y2, y2]
        constraint_matrix.append(constraint_1)
        constraint_matrix.append(constraint_2)

    constraint_matrix = np.array(constraint_matrix)
    u, s, v_t = np.linalg.svd(constraint_matrix)
    least_squares = v_t[-1] # last column of V
    H = least_squares.reshape((3, 3))
    H = H / H[2, 2]
    return H

# Interactive  homography 
Here we will interactively plot the homography between two views of a plane

In [5]:
%matplotlib notebook
from notebook_functions import (plot_camera_wireframe,
                                project_points_to_picture, 
                                get_rotation_matrix)
import ipyvolume as ipv
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib

In [6]:
class Camera:
    def __init__(self, color='blue'):
        self.extrinsics = np.identity(4) 
        self.color = color
        
    def update_extrinsics(self, x, y, z, rot_x, rot_y, rot_z):
        rot = get_rotation_matrix(rot_x, rot_y, rot_z)
        trans = -np.array([x, y, z])
        self.extrinsics =  np.vstack([np.hstack([rot, trans[:, None]]), [0, 0, 0, 1.0]])
    
    def change_of_coords(self, coc): 
        self.extrinsics = coc @ self.extrinsics
    
    @property
    def trans(self):
        return self.extrinsics[:3, 3]
    
    @property 
    def rot(self): 
        return self.extrinsics[:3, :3]
        
    @property
    def inv_extrinsics(self): 
        return np.linalg.pinv(self.extrinsics)
    
    @property
    def center(self):
        return (-self.rot.T) @ self.trans

    @property
    def border(self): 
        inv_extrinsics = self.inv_extrinsics
        lt = inv_extrinsics @ [-1 , 2/3 , 2/3, 1]
        rt = inv_extrinsics @ [1  , 2/3 , 2/3, 1]
        lb = inv_extrinsics @ [-1 , -2/3, 2/3, 1]
        rb = inv_extrinsics @ [1  , -2/3, 2/3, 1]
        
        return [lt, rt, lb, rb] 
    
    def project(self, objects):
        projected = self.extrinsics @ objects
        projected /= projected[2]
        return projected[:3]
   
    def plot_wireframe(self): 
        x, y, z = self.center[:3].flatten().astype(float)
        [lt, rt, lb, rb] = self.border 
        
        # Connect camera projective center to wireframe extremities
        self.p1 = ipv.plot([x, lt[0]], [y, lt[1]], [z, lt[2]], color=self.color)
        self.p2 = ipv.plot([x, rt[0]], [y, rt[1]], [z, rt[2]], color=self.color)
        self.p3 = ipv.plot([x, lb[0]], [y, lb[1]], [z, lb[2]], color=self.color)
        self.p4 = ipv.plot([x, rb[0]], [y, rb[1]], [z, rb[2]], color=self.color)

        # Connect wireframe corners with a rectangle
        self.p5 = ipv.plot([lt[0], rt[0]], [lt[1], rt[1]], [lt[2], rt[2]], color=self.color)
        self.p6 = ipv.plot([rt[0], rb[0]], [rt[1], rb[1]], [rt[2], rb[2]], color=self.color)
        self.p7 = ipv.plot([rb[0], lb[0]], [rb[1], lb[1]], [rb[2], lb[2]], color=self.color)
        self.p8 = ipv.plot([lb[0], lt[0]], [lb[1], lt[1]], [lb[2], lt[2]], color=self.color)
        
        self.p9 = ipv.scatter(np.array([x]), np.array([y]), np.array([z]), size=1, marker="sphere", color=self.color)

    def update_wireframe(self):
        [lt, rt, lb, rb] = self.border 

        with self.p1.hold_sync(), self.p2.hold_sync(), self.p3.hold_sync(), self.p4.hold_sync(), self.p5.hold_sync(), self.p6.hold_sync(), self.p7.hold_sync(), self.p8.hold_sync(), self.p9.hold_sync():
            x, y, z = self.center[:3].flatten().astype(float)
            
            [lt, rt, lb, rb] = self.border 
            self.p1.x, self.p1.y, self.p1.z = [x, lt[0]], [y, lt[1]],  [z, lt[2]]
            self.p2.x, self.p2.y, self.p2.z = [x, rt[0]], [y, rt[1]],  [z, rt[2]]
            self.p3.x, self.p3.y, self.p3.z = [x, lb[0]], [y, lb[1]], [z, lb[2]]
            self.p4.x, self.p4.y, self.p4.z = [x, rb[0]], [y, rb[1]], [z, rb[2]]
            self.p5.x, self.p5.y, self.p5.z = [lt[0], rt[0]], [lt[1], rt[1]], [lt[2], rt[2]]
            self.p6.x, self.p6.y, self.p6.z = [rt[0], rb[0]], [rt[1], rb[1]], [rt[2], rb[2]]
            self.p7.x, self.p7.y, self.p7.z = [rb[0], lb[0]], [rb[1], lb[1]], [rb[2], lb[2]]
            self.p8.x, self.p8.y, self.p8.z = [lb[0], lt[0]], [lb[1], lt[1]], [lb[2], lt[2]]
            self.p9.x, self.p9.y, self.p9.z = np.array([x]), np.array([y]), np.array([z])    

In [12]:
import ipyvolume as ipv
import numpy as np
import matplotlib
from notebook_functions import (plot_camera_wireframe,
                                update_camera_wireframe,
                                project_points_to_picture, 
                                get_rotation_matrix,
                                triangulate)
import copy

rot_x_1 = FloatSlider(min=-np.pi, max=np.pi, step=0.005, description=r'\(\theta_x\)')
rot_y_1 = FloatSlider(min=-np.pi, max=np.pi, step=0.005, description=r'\(\theta_y\)')
rot_z_1 = FloatSlider(min=-np.pi, max=np.pi, step=0.005, description=r'\(\theta_z\)')
rot_x_2 = FloatSlider(min=-np.pi, max=np.pi, step=0.005, description=r'\(\theta_x\)')
rot_y_2 = FloatSlider(min=-np.pi, max=np.pi, step=0.005, description=r'\(\theta_y\)')
rot_z_2 = FloatSlider(min=-np.pi, max=np.pi, step=0.005, description=r'\(\theta_z\)')

x_trans_1 = FloatSlider(min=-5, max=5, step=0.005, description=r'x translation  \(t_x\)')
y_trans_1 = FloatSlider(min=-5, max=5, step=0.005,  description=r'y translation \(t_y\)')
z_trans_1 = FloatSlider(min=-5, max=5, step=0.005,  description=r'y translation \(t_y\)')
x_trans_2 = FloatSlider(min=-5, max=5, step=0.005, description=r'x translation  \(t_x\)')
y_trans_2 = FloatSlider(min=-5, max=5, step=0.005,  description=r'y translation \(t_y\)')
z_trans_2 = FloatSlider(min=-5, max=5, step=0.005,  description=r'y translation \(t_y\)')

sliders_1 = [rot_x_1, rot_y_1, rot_z_1, x_trans_1, y_trans_1, z_trans_1]
sliders_2 = [rot_x_2, rot_y_2, rot_z_2, x_trans_2, y_trans_2, z_trans_2]
all_sliders = sliders_1 + sliders_2

camera_view_1 = get_quad_fig('Camera 1 View')
camera_view_2 = get_quad_fig('Camera 2 View')
homography_fig = get_quad_fig('Camera 2 according to homography')

ipv_fig = ipv.figure()
ipv.xlim(-5, 5)
ipv.ylim(-5, 5)
ipv.zlim(-5, 5)

x_coords = np.array([-1, 1, 1, -1]).astype(float)
y_coords = np.array([1, 1, -1, -1]).astype(float)
z_coords = np.array([1, 1, 1, 1]).astype(float)
objects = np.vstack([x_coords, y_coords, z_coords, np.ones_like(x_coords)])
colors = [matplotlib.colors.to_rgba(name) for name in ['salmon', 'lawngreen', 'deepskyblue', 'orange']]
ipv.scatter(x_coords, y_coords, z_coords, marker='sphere', color=colors)
ipv.plot(np.pad(x_coords, 1, 'wrap'), np.pad(y_coords, 1, 'wrap'), np.pad(z_coords, 1, 'wrap'), color='deepskyblue')

cam_1 = Camera('blue')
cam_2 = Camera('green')
cam_1.plot_wireframe()
cam_2.plot_wireframe()

def update_homography(): 
    """ Updates the 2D scatter plot and lines visualizing the homography """
    scatter = homography_fig.marks[0]
    lines = homography_fig.marks[1]
        
    points_1 = cam_1.project(objects)
    points_2 = cam_2.project(objects)
    homography = compute_homography(points_1, points_2)
    homography_html.value = get_value_matrix_html(homography)
    
    points_2_h = homography @ points_1[:3]
    points_2_h = points_2_h / points_2_h[2]
    scatter.x, scatter.y = -points_2_h[0], points_2_h[1]
    lines.x, lines.y = -points_2_h[0], points_2_h[1]
    
    return homography 

homography = None

def on_slider_drag(*args):
    """ Sets appropriate camera position. """
    global homography
    if sliders_tab.selected_index == 0: 
        cam = cam_1
        scatter = camera_view_1.marks[0]
        lines = camera_view_1.marks[1]
        sliders = sliders_1
    else: 
        cam = cam_2
        scatter = camera_view_2.marks[0]
        lines = camera_view_2.marks[1]
        sliders = sliders_2

    rot_x, rot_y, rot_z, x, y, z = [slider.value for slider in sliders]
    cam.update_extrinsics(x, y, z, rot_x, rot_y, rot_z)
    cam.update_wireframe()
    
    # project to image
    with scatter.hold_sync(), lines.hold_sync():
        xs, ys, zs = cam.project(objects)
        scatter.x, scatter.y = -xs, ys 
        lines.x, lines.y = -xs, ys 
    
    homography = update_homography()
        
for slider in all_sliders: 
    slider.observe(on_slider_drag)
    
slider_box_1 = VBox(sliders_1[:3])
slider_box_2 = VBox(sliders_1[3:])
slider_box_3 = VBox(sliders_2[:3])
slider_box_4 = VBox(sliders_2[3:])
sliders_ui_1 = HBox([slider_box_1, slider_box_2], layout=Layout(margin='20px'))
sliders_ui_2 = HBox([slider_box_3, slider_box_4], layout=Layout(margin='20px'))
sliders_tab = Tab([sliders_ui_1, sliders_ui_2], _titles={0:'Camera 1', 1:'Camera 2'})
homography_html = HTML(value=get_value_matrix_html(np.zeros([3, 3])))

space_vis = HBox([ipv_fig, camera_view_1, camera_view_2, homography_fig])
ui = VBox([sliders_tab, space_vis, homography_html])
display(ui)

## Homography decomposition

### Symbols
* $\mathbf{n}$ - Normal to the plane
* $\mathbf{R}$ - Rotation that brings cam 1 to cam 2 orientation
* $\mathbf{t}$ - Translation that brings cam 1 to cam 2 position
* $d$ - Distance from camera one to the plane
* $\mathbf{A} = d \mathbf{R} + \mathbf{tn}^t$
* $\mathbf{X_2} = \mathbf{A} \mathbf{X_1}$
* $\mathbf{X_2} = (\mathbf{R} + \frac{\mathbf{tn}^t}{d}) \mathbf{X_1})$
* SVD: $\mathbf{A} = \mathbf{U} \mathbf{\Lambda} \mathbf{V^t} $ 
* $\mathbf{\Lambda} = d' \mathbf{R'} + \mathbf{t'n'^t}$
* $\mathbf{\Lambda} = \begin{bmatrix} \sigma_1 & 0        & 0 \\
                                      0        & \sigma_2 & 0 \\
                                      0        & 0        & \sigma_3 \end{bmatrix}$ 
                    
Relation between variables and primes     
variables:      
* $\mathbf{R} = \mathbf{sUR'V^t}$
* $\mathbf{t} = \mathbf{Ut'}$
* $\mathbf{n} = \mathbf{Vn'}$
* $d = sd'$
* $s = det \hspace{0.1em}\mathbf{U} \hspace{0.2em}det\hspace{0.1em} \mathbf{V}$

primes: 
* $\mathbf{R'} = s \mathbf{U^tRV}$
* $\mathbf{t'} = \mathbf{U^t t}$
* $\mathbf{n'} = \mathbf{V^t n}$
* $d' = sd$

 

### Solutions
solutions for singular values that are all unique (multiplicity $\Lambda$= 3)     
     
$\epsilon_1 \pm 1$    
$\epsilon_2 \pm 1$    

$n_1 = \epsilon_1 \frac{\sqrt{\sigma_1^2 - \sigma_2^2}}{\sigma_1^2 - \sigma^2_3}$      
$n_2 = 0 $     
$n_3 = \epsilon_3 \frac{\sqrt{\sigma_2^2 - \sigma_3^2}}{\sigma_1^2 - \sigma^2_3}$    

<br> 
<br>
 
Case $d' > 0$:       
 

$\mathbf{R'} = \begin{bmatrix} \cos \theta & 0 & - \sin \theta \\
                               0           & 1 & 0             \\
                               \sin \theta & 0 & \cos \theta
               \end{bmatrix}$     
               
$ \sin \theta = (\sigma_1 - \sigma_3) \frac{n_1 n_3}{\sigma_2} $        

$ \cos \theta = \frac{\sigma_1 n^2_3 + \sigma_3 n^2_1}{\sigma_2} $     
   
$\mathbf{t'} = (\sigma_1 - \sigma_3) \begin{bmatrix} n_1 \\ 0 \\ n_3 \end{bmatrix}$

<br>
<br>  

 
Case $d' < 0$:


$\mathbf{R'} = \begin{bmatrix} \cos \theta & 0  & \sin \theta \\
                               0           & -1 & 0           \\
                               \sin \theta & 0  & - \cos \theta
               \end{bmatrix}$        
               
$ \sin \theta = \frac{\sigma_1 + \sigma_3}{\sigma_2}n_1 n_3$       
 
$ \cos \theta = \frac{\sigma_3 n_1^2 - \sigma_1 n_3^2}{\sigma_2}$

$\mathbf{t'} = (\sigma_1 + \sigma_3) \begin{bmatrix} n_1 \\ 0 \\ n_3 \end{bmatrix}$


In [13]:
def decompose_homography(homography): 
    u, s, v_t = np.linalg.svd(homography)
    v = v_t.T
    s1, s2, s3 = s  
    
    s = np.linalg.det(u) * np.linalg.det(v)
    
    if not (s1 != s2  and s2 != s3): 
        print('singular values are not unique') 
        return None
    
    n1 = np.sqrt((s1**2 - s2**2) / (s1**2 - s3**2))
    n2 = 0 
    n3 = np.sqrt((s2**2 - s3**2) / (s1**2 - s3**2))

    sign_permutations = [(1, 1), (1, -1), (-1, 1), (-1, -1)]

    rots = list()
    transs = list()
    normals = list()
    distances = list()
    change_of_coords = list()
    
    for d_positive in (True, False):
        for (e1, e3) in sign_permutations: 
    
            n1e = e1 * n1
            n3e = e3 * n3
            normal_prime = np.array([n1e, 0, n3e])
            
            if d_positive:
                distance = s * s2
                sin_theta = (s1 - s3) * ((n1e * n3e) / s2)
                cos_theta = (s1 * n3e**2 + s3 * n1e**2) / s2
                
                rot_prime = np.array([[cos_theta, 0, -sin_theta],
                                      [0        , 1, 0         ],
                                      [sin_theta, 0, cos_theta ]])
                
                trans_prime = (s1 - s3) * np.array([n1e, 0 , n3e])
                
            else:
                distance =  s * -s2
                sin_theta = ((s1 + s3) / s2) * n1e * n3e
                cos_theta = (s3 * n1e**2 - s1 * n3e**2) / s2

                rot_prime = np.array([[cos_theta, 0 , sin_theta  ],
                                      [0        , -1, 0          ],
                                      [sin_theta, 0 , -cos_theta ]])

                trans_prime = (s1 + s3) * np.array([n1e, 0 , n3e])
                
                
            
            rot = s * u @ rot_prime @ v.T
            trans = u @ trans_prime
            normal = v @ normal_prime

            rots.append(rot)
            transs.append(trans) 
            normals.append(normal)
            distances.append(distance)

            coc = np.vstack([np.hstack([rot, trans[:, None]]), [0, 0, 0, 1]])
            change_of_coords.append(coc)
        
    return change_of_coords, normals, distances

# Apply decomposition and test results

In [14]:
change_of_coords, normals, distances = decompose_homography(homography)

test_1_scores = list()
points_1 = cam_1.project(objects)

# Test 1. Does the homography lead to a positive z coordinate
for coc, plane_distance in zip(change_of_coords, distances): 
    points_2 = homography @ points_1
    zs = points_2[2] / plane_distance
    visible = (zs > 0).sum()
    test_1_scores.append(visible)

# Sort decompositions highest to lowest.
decompositions = sorted(zip(test_1_scores, change_of_coords, normals, distances), reverse=True, key=lambda x: x[0])
# Keep only the top 4 scoring decompositions
decompositions = decompositions[:4]
_, change_of_coords, normals, distances = list(zip(*decompositions))

ones = np.ones_like(objects[0])[None, :]
points_1 = np.vstack([points_1, ones])
test_2_scores = list()


# Test 2. Are the points observed on the front of the plane? (Does the normal of the plane point in the same direction as the unprojected point) 
for coc, normal, plane_distance in zip(change_of_coords, normals, distances):  
    vecs = cam_1.inv_extrinsics @ points_1
    dot_products = np.dot(normal, vecs[:3]) / plane_distance
    test_2_scores.append((dot_products > 0).sum())
    
decompositions = sorted(zip(test_2_scores, change_of_coords, normals, distances), reverse=True, key=lambda x: x[0])
# Keep only the top 2 scoring decompositions
decompositions = decompositions[:2]
test_2_scores, change_of_coords, normals, distances = list(zip(*decompositions))

## If the ratio is good, keep the solution
# ratio = test_2_scores[1] / test_2_scores[0]
# print(ratio)
# # if ratio < 0.9: 
# coc[:3, 3] = coc[:3, 3] / np.linalg.norm(coc[:3, 3])
# coc[:3, 3] *= np.linalg.norm(cam_1.trans - cam_2.trans)
trans_cam = Camera('red')
trans_cam.extrinsics = cam_1.extrinsics # Make a copy of cam 1
trans_cam.change_of_coords(coc)
trans_cam.plot_wireframe()

# Conclusion

The decomposition is a succes, the pose of ground truth camera 2 is the same as the decomposition.
But notice how the scale of the translation is unknown, as also with the fundamental matrix decomposition (in notebook 3).

## Sources

1. [Zisserman, Richard Hartley Andrew. "Multiple view geometry in computer vision." 2004.](https://www.robots.ox.ac.uk/~vgg/hzbook/)
2. [Elan Dubrofsky, "Homography Estimation." 2009.](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.186.4411)


Todo, check this out https://users.cecs.anu.edu.au/~hartley/Papers/CVPR99-tutorial/tutorial.pdf